Use LLM to evalute the generate question and filter the question

In [1]:
question_groundedness_critique_prompt = """
你将获得上下文内容和问题。
您的任务是提供“评分”，以评估在给定上下文中明确回答给定问题的能力。
按照 1 到 5 的分数给出答案，其中 1 表示该问题在给定上下文的情况下根本无法回答，5 表示该问题可以根据上下文清楚、明确地回答。

请按照如下方式提供答案：

答案::: 
评估理由:(你评分的理由，以文本形式) 
评分:(你的评分，以 1 到 5 之间的数字表示）

你必须在答案中提供“评估理由”和“评分”。

下面是问题和上下文。

问题:{question}\n
上下文:{context}\n
答案::: """

question_relevance_critique_prompt = """
你将收到一个问题。
你的任务是提供“评分”，表示该问题对于一个普通公民对理解政策文件有多大用处。
按照 1 到 5 的范围给出答案，其中 1 表示该问题根本没有用，5 表示该问题非常有用。

请按照如下方式提供你的答案： 

答案::: 
评估理由:(你评分的理由，以文本形式) 
评分:(你的评分，以 1 到 5 之间的数字表示）

你必须在答案中提供“评估理由”和“评分”。

下面是问题。

问题:{question}\n
答案::: """

question_standalone_critique_prompt = """
您将收到一个问题。
您的任务是提供一个“评分”，表示该问题与上下文的独立程度。
按照 1 到 5 的范围给出答案，其中 1 表示问题依赖于其他信息才能理解，5 表示问题本身有意义。
例如，如果问题涉及特定设置，例如“在上下文中”或“该文件”，则评级必须为 1。
这些问题可以包含一些特定的指代，例如我市，但仍然是 5：必须让有权访问文件的人员清楚问题的内容。

例如：“该文件实施的背景是什么？” 应该评分为1，因为隐含地提到了上下文，因此该问题并不是独立于上下文的。

请按如下方式提供您的答案：

答案::: 
评估理由:(你评分的理由，以文本形式) 
评分:(你的评分，以 1 到 5 之间的数字表示）

你必须在答案中提供“评估理由”和“评分”。

下面是问题。

问题:{question}\n
答案::: """

In [ ]:
import os 
import pandas as pd
path = r'RAG_PLUS\generate_qa_couples\batchQA_output'

path_list = os.listdir(path)
for index, doc in enumerate(path_list):
    real_path = os.path.join(path, doc)

    try:
        df = pd.read_csv(real_path)
    except:
        continue

    outputs = df.to_dict(orient='records')

    print(f"{index}doc---Generating critique for each QA couple...")
    for output in tqdm(outputs):
        time.sleep(2)
        evaluations = {
            "groundedness": ask_glm(question_groundedness_critique_prompt.format(context=output['context'], question=output['question']))['choices'][0]['message']['content'],
            "relevance": ask_glm(question_relevance_critique_prompt.format(question=output['question']))['choices'][0]['message']['content'],
            "standalone": ask_glm(question_standalone_critique_prompt.format(question=output['question']))['choices'][0]['message']['content']
        }
        try:
            for criterion, evaluation in evaluations.items():
                evaluation = evaluation.strip().replace(':','').replace("：",'').replace("-", '').replace("\n", '')
                print(evaluation)
                score, eval_res = (
                    int(evaluation.split("评分")[-1].strip()),
                    evaluation.split("评分")[-2].split("评估理由")[1].strip(),
                )
                output.update(
                    {
                        f"{criterion}_score": score,
                        f"{criterion}_eval": eval_res,
                    }
                )
        except Exception as e:
            continue

    df = pd.DataFrame(outputs)
    df.to_csv(f'./glm_evaluate_q/{index}doc.csv', index=False, encoding='utf-8-sig')    


Concat and filter the rating data

In [ ]:
# concat 所有的评分数据
import pandas as pd
import os
dir_path = r"C:\Users\Lenovo\Desktop\CUHK\CUHK_2024\Course\Project\RAG_Plus\glm_evaluate_q"
doc_frames = []
doc_path_list = os.listdir(dir_path)
for doc_name in doc_path_list:
    doc_path = os.path.join(dir_path, doc_name)
    doc_csv = pd.read_csv(doc_path)
    doc_frames.append(doc_csv)

combined_df = pd.concat(doc_frames, ignore_index=True)
combined_df.to_csv("evaluate_qa_pair.csv", index=False)

    

In [ ]:
import datasets
generated_questions = pd.read_csv("evaluate_qa_pair.csv")
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 3)
]

eval_dataset = datasets.Dataset.from_pandas(generated_questions, split='train', preserve_index=False)
generated_questions.info()